In [1]:
from batchcreator import DataGenerator, get_list_IDs
from datetime import datetime
import tensorflow as tf
from model_builder import build_generator, GAN
import numpy as np

import matplotlib.pyplot as plt

In [2]:
model = GAN(rnn_type='LSTM')
model.build((None,5, 768, 700, 1))
model.compile()
model.summary()

Model: "gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Discriminator (Functional)   (None, 1)                 394753    
_________________________________________________________________
Generator (Functional)       (None, 1, 384, 350, 1)    28904233  
Total params: 29,298,992
Trainable params: 29,298,986
Non-trainable params: 6
_________________________________________________________________


In [3]:
model = GAN(rnn_type='GRU')
model.build((None,5, 768, 700, 1))
model.compile()
model.summary()

Model: "gan_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Discriminator (Functional)   (None, 1)                 394753    
_________________________________________________________________
Generator (Functional)       (None, 1, 384, 350, 1)    22168105  
Total params: 22,562,864
Trainable params: 22,562,858
Non-trainable params: 6
_________________________________________________________________


In [4]:
batch_size =1

# Create list of IDs to retrieve
x_seq_size=5
y_seq_size=1

start_dt = datetime(2019,6,4,0,0)
end_dt =  datetime(2019,6,6,0,30)
# Exclude samples were there is no rain in the input
filter_no_rain = True
list_IDs = get_list_IDs(start_dt, end_dt, x_seq_size, y_seq_size, filter_no_rain=filter_no_rain)
print(list_IDs[-1])
print(len(list_IDs))

(['201906052355', '201906060000', '201906060005', '201906060010', '201906060015'], ['201906060020'])
109


In [5]:
# Test generator
generator = DataGenerator(list_IDs, batch_size=batch_size, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, norm_method='minmax')
for i in generator:
    x,y = i
    print(x.shape, y.shape)

(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)
(1, 5, 768, 700, 1) (1, 1, 384, 350, 1)


In [6]:
model

In [ ]:
generator = DataGenerator(list_IDs, batch_size=batch_size, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, norm_method='minmax')
hist = model.fit(generator, epochs=3)

Epoch 1/3
 17/109 [===>..........................] - ETA: 11:46 - d_loss: 0.6931 - g_loss: 0.6931 - mse_loss: 0.2495

In [ ]:
model.save("gan_model")

In [ ]:
model.generator.save("generator")

In [ ]:
# Test generator
start_dt = datetime(2019,6,6,0,0)
end_dt =  datetime(2019,6,6,0,5)

list_IDs = get_list_IDs(start_dt, end_dt, x_seq_size, y_seq_size, filter_no_rain=filter_no_rain)
generator = DataGenerator(list_IDs, batch_size=1, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, normalize_y=False)
for i in generator:
    x,y = i
    print(x.shape, y.shape)

In [ ]:
y_pred = model.generator(x)
y_pred.shape

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.squeeze(y))
plt.colorbar()
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(np.squeeze(y_pred))
plt.colorbar()
plt.show()

In [ ]:
plt.hist(y_pred.numpy().flatten())

In [ ]:
model.discriminator(y_pred)